# Imports

In [ ]:
# Move to the root
import os
cwd = os.getcwd()
if os.path.basename(cwd) != "cv-in-farming":
    os.chdir("../")
print(os.getcwd())

## Libraries

In [ ]:
import random
import os
import time

import matplotlib

from ipywidgets import *
from IPython.display import display

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Import Utility Functions 

Importing from [helpers.py](../utils/helpers.py).

In [ ]:
from utils.helpers import *

## Import Image Processing Functions

Importing from [image_processing.py](../src/image_processing.py).

In [ ]:
from src.image_processing import *

# Load Frames

## Load all frames

In [ ]:
folders = ["discarded_front(20201112_140127)"]
triplets_per_folder = {}

for folder in folders:
    triplets_per_folder[folder] = []
    files = sorted(os.listdir(folder), key=lambda file: int(file.split("_")[0]))
    rgb_im_files = list(filter(lambda file: "rgb" in file, files))
    depth_im_files = list(filter(lambda file: "depth.png" in file, files))
    depth_arr_files = list(filter(lambda file: "npy" in file, files))
    num_files = len(rgb_im_files)
    
    for i in range(num_files):
        rgb_im_path = os.path.join(folder, rgb_im_files[i])
        depth_im_path = os.path.join(folder, depth_im_files[i])
        depth_arr_path = os.path.join(folder, depth_arr_files[i])

        rgb_img = cv2.imread(rgb_im_path, cv2.IMREAD_COLOR)     # BGR image
        depth_img = cv2.imread(depth_im_path, cv2.IMREAD_COLOR) # BGR image
        depth_arr = np.load(depth_arr_path)
        
        triplets_per_folder[folder].append((rgb_img, depth_img, depth_arr))
    
print("Number of folders:", len(triplets_per_folder))
for i, k in enumerate(folders):
    print("{}) Loaded {} frames from {}".format(i, len(triplets_per_folder[k]), k))

## Pick one frame

In [ ]:
# folder_indices = list(range(8))
# folder_idx = random.choice(folder_indices)
# print("Folder choice:", folder_idx)
folder_idx = 0
folder = folders[folder_idx]

file_indices = list(range(0, 120))
frame_idx = random.choice(file_indices)
print("Frame choice:", frame_idx)
# frame_idx = 0

print("Fetching frame: {} from folder: {}".format(frame_idx+1, folder))
rgb_img, depth_img, depth_arr = triplets_per_folder[folder][frame_idx]

show_image_pairs(rgb_img, depth_img)

# frame: 13 from folder: frames/20201112_133434 (3) -> "Grayscale", "Sobel Vertical", "Threshold", "Hough Line"
# frame: 13 from folder: frames/20201112_140127 (6) -> "Grayscale", "Sobel Vertical", "Spatial Threshold", "Threshold", "Hough Line"

# Create UI Elements

## Define UI Callback Functions

Register a callback for configuration of image processing methods here if needed.

In [ ]:
def dilation_config(ksize=5):
    return {"ksize": ksize}
    
def erosion_config(ksize=5):
    return {"ksize": ksize}

def opening_config(ksize=5):
    return {"ksize": ksize}
    
def closing_config(ksize=5):
    return {"ksize": ksize}

def threshold_config(thresh=127, maxval=255, type=0):
    return {"thresh": thresh, "maxval": maxval, "type": type}

def adaptive_threshold_config(maxValue, adaptiveMethod, thresholdType, blockSize, C):
    return {"maxValue": maxValue, "adaptiveMethod": adaptiveMethod, "thresholdType": thresholdType, "blockSize": blockSize, "C": C}

def spatial_threshold_config(min_x=0, max_x=640, min_y=0, max_y=480):
    return {"range_x": (min_x, max_x), "range_y": (min_y, max_y)}

def avg_blur_config(ksize):
    return {"ksize": ksize}

def median_blur_config(ksize):
    return {"ksize": ksize}

def bilateral_filter_config(d, sigmaColor, sigmaSpace):
    return {"d": d, "sigmaColor": sigmaColor, "sigmaSpace": sigmaSpace}

def gaussian_blur_config(ksize=5, sigmaX=10):
    return {"ksize": (ksize, ksize), "sigmaX": sigmaX}

def laplacian_config(ksize=5):
    return {"ksize": ksize}
    
def sobelv_config(ksize=5):
    return {"ksize": ksize}
    
def sobelh_config(ksize=5):
    return {"ksize": ksize}

def canny_config(threshold1=100, threshold2=200, apertureSize=5):
    return {"threshold1": threshold1, 
            "threshold2": threshold2, 
            "apertureSize": apertureSize}

def config_shi_tomasi(blockSize=25, 
                      maxCorners=20, 
                      qualityLevel=0.75, 
                      minDistance=20):
    return {"blockSize": blockSize, 
            "maxCorners": maxCorners, 
            "qualityLevel": qualityLevel, 
            "minDistance": minDistance}
    
def config_dsift(top_n=1000, step=1, size=5):
    return {"top_n": top_n, "step": step, "size": size}

def config_sift(peak_thresh=0, edge_thresh=10, norm_thresh=0, window_size=2):
    return {"peak_thresh": peak_thresh, 
            "edge_thresh": edge_thresh, 
            "norm_thresh": norm_thresh, 
            "window_size": window_size}

def hough_line_config(average=True,
                      adaptive_thresh=True, # use half of the max vote as threshold
                      num_lines=5,     # take num_lines top voted lines
                      min_theta=0,     # minimum angles to accept a line
                      max_theta=180,   # maximum angles to accept a line
                      min_rho=-800, # minimum rho to accept a line
                      max_rho=800,  # maximum rho to accept a line
                      threshold=200,   # minimum votes required to accept a line
                      theta_res=1,     # theta_res: 2 -> unit angle: 0.5
                      min_distance=9,  # take lines separated with at least min_distance pixels
                      min_angle=10):   # take lines separated with at least min_angle degrees
    return {"average": average,
            "adaptive_thresh": adaptive_thresh,
            "num_lines": num_lines,
            "min_theta": min_theta,
            "max_theta": max_theta,
            "min_rho": min_rho,
            "max_rho": max_rho,
            "threshold": threshold,
            "theta_res": theta_res,   
            "min_distance": min_distance,
            "min_angle": min_angle}

## Define Name-UI Mappings

Bind names to the UI elements. Define new UI elements and name-UI element mappings here if needed.

In [ ]:
# Defining UI elements for configurations
dilation_ui = lambda: interactive(dilation_config, ksize=(1,20))
erosion_ui = lambda: interactive(erosion_config, ksize=(1,20))
opening_ui = lambda: interactive(opening_config, ksize=(1,20))
closing_ui = lambda: interactive(closing_config, ksize=(1,20))
threshold_ui = lambda: interactive(threshold_config, thresh=(0.0,255.0,0.5), maxval=(0.0,255.0,0.5), 
                           type={"Binary": cv2.THRESH_BINARY,
                                 "Binary Inverted": cv2.THRESH_BINARY_INV,
                                 "Trunctate": cv2.THRESH_TRUNC,
                                 "To Zero": cv2.THRESH_TOZERO,
                                 "To Zero Inverted": cv2.THRESH_TOZERO_INV})
adaptive_threshold_ui = lambda: interactive(adaptive_threshold_config, 
                                       maxValue=(0,255), 
                                       adaptiveMethod={
                                           'Adaptive Gaussian Thresholding': cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                           'Adaptive Mean Thresholding': cv2.ADAPTIVE_THRESH_MEAN_C},
                                       thresholdType={"Binary": cv2.THRESH_BINARY,
                                                      "Binary Inverted": cv2.THRESH_BINARY_INV,
                                                      "Trunctate": cv2.THRESH_TRUNC,
                                                      "To Zero": cv2.THRESH_TOZERO,
                                                      "To Zero Inverted": cv2.THRESH_TOZERO_INV},
                                       blockSize=(0,20),
                                       C=(0,20))
spatial_threshold_ui = lambda: interactive(spatial_threshold_config, 
                                           min_x=(0,1000), max_x=(0,1000),
                                           min_y=(0,1000), max_y=(0,1000))
avg_blur_ui = lambda: interactive(avg_blur_config, ksize=(1,20))
median_blur_ui = lambda: interactive(median_blur_config, ksize=(1,20))
bilateral_filter_ui = lambda: interactive(bilateral_filter_config, d=(1,10), sigmaColor=(0,255), sigmaSpace=(0,200))
gaussian_blur_ui = lambda: interactive(gaussian_blur_config, ksize=(1,20), sigmaX=(1,20))
laplacian_ui = lambda: interactive(laplacian_config, ksize=(1,20))
sobelv_ui = lambda: interactive(sobelv_config, ksize=(1,20))
sobelh_ui = lambda: interactive(sobelh_config, ksize=(1,20))
canny_ui = lambda: interactive(canny_config, threshold1=(0,500), threshold2=(0,500), apertureSize=(3, 7))
shi_tomasi_ui = lambda: interactive(config_shi_tomasi, 
                                    blockSize=(0,100), 
                                    maxCorners=(0,200), 
                                    qualityLevel=(0.0,1.0,0.1),
                                    minDistance=(0,200))
dsift_ui = lambda: interactive(config_dsift, 
                               top_n=(0,200), 
                               step=(1,100), 
                               size=(1,100))
sift_ui = lambda: interactive(config_sift, 
                              peak_thresh=(0,200), 
                              edge_thresh=(0,200), 
                              norm_thresh=(0,200), 
                              window_size=(1,100))
hough_line_ui = lambda: interactive(hough_line_config,
                                    num_lines=(1,500),
                                    min_theta=(0,180),
                                    max_theta=(0,180),
                                    min_rho=(-800,800),
                                    max_rho=(-800,800),
                                    threshold=(0,100000),
                                    theta_res=(1,5),
                                    min_distance=(0,1000),
                                    min_angle=(0,180))

# Binding names to the configuration UI elements
preproc_ui = {
    "Grayscale": None,
    "Contrast": None,
    "Dilation": dilation_ui,
    "Erosion": erosion_ui,
    "Opening": opening_ui,
    "Closing": closing_ui,
    "ROI Threshold": None,
    "Threshold": threshold_ui,
    "Adaptive Threshold": adaptive_threshold_ui,
    "Spatial Threshold": spatial_threshold_ui,
    "Average Blur": avg_blur_ui,
    "Median Blur": median_blur_ui,
    "Bilateral Filter": bilateral_filter_ui,
    "Gaussian Blur": gaussian_blur_ui,
    "Laplacian": laplacian_ui,
    "Sobel Vertical": sobelv_ui,
    "Sobel Horizontal": sobelh_ui,
}

detect_ui = {
    "Canny Edges": canny_ui,
    "Contour": None,
    "Shi Tomasi": shi_tomasi_ui,
    "Dense SIFT": dsift_ui,
    "SIFT": sift_ui,
    "Hough Line": hough_line_ui,
}

ui_elements = {**preproc_ui, **detect_ui}

# Create New Experiment

## Select Methods 

In [ ]:
style = {
    'description_width': 'initial'
}

func_stack = []
add_buttons = []
log = Output(layout={'margin': '0px 50px 0px 50px', 'width': "200px"})

for item in ui_elements:
    button = Button(
        description=item,
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    )
    button.on_click(lambda e: push(func_stack, log, e))
    add_buttons.append(button)

pop_button = Button(
    description="Pop",
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
)
pop_button.on_click(lambda _: pop(func_stack, log))
    
menu1 = HBox([VBox(add_buttons), log, pop_button])

display(menu1)

## Configure Selected Methods

In [ ]:
ui_instances = []
for i, k in enumerate(func_stack):
    if ui_elements[k]:
        print("{} {}:".format(i, k))
        ui_instance = ui_elements[k]()
        display(ui_instance)
        ui_instances.append(ui_instance)
    else:
        ui_instances.append(None)

## Apply Configured Methods

In [ ]:
input_image = depth_img.copy() # rgb_img
roi_config = {}
NUM_CORNERS = 5

idx = -1
try:
    idx = func_stack.index("ROI Threshold")
except Exception:
    print("[Info]: ROI Threshold is not selected.")

if idx >= 0:
    matplotlib.use("Qt5Agg")
    roi_config["roi_corners"] = set_roi(input_image, NUM_CORNERS)
    print("[Info]: ROI Threshold is configured.")
    %matplotlib inline

In [ ]:
pprint(roi_config)

In [ ]:
config_stack = []
# Update internal state
for ui_instance in ui_instances:
    config = {}
    if ui_instance:
        config = ui_instance.result
    config_stack.append(config)
if roi_config:
    config_stack[idx] = roi_config

input_image = depth_img.copy() # rgb_img
output = apply_functions(input_image.copy(), func_stack, config_stack)
# show_shapes(input_image.copy(), output, shapeIdx=0, cmap=None)

## Store Current Experiment

In [ ]:
text = Text(
    value='exp.json',
    placeholder='Enter a filename',
    description='Filename:',
    disabled=False
)


button = Button(
    description='Store',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
)

mini_form = HBox([text, button])

button.on_click(lambda _: save_config(text.value, func_stack, config_stack))

display(mini_form)

# Load Previous Experiment from File

In [ ]:
# TODO: Ability to continue with loaded experiment
config_stack = []
uploader_ui = FileUpload(
    accept='.json', # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

uploader_ui.observe(lambda _: load_config(*uploader_ui.data, func_stack, config_stack), names='_counter')

display(uploader_ui)

In [ ]:
exps = []

def load_exp(json_contents, exps):
    for json_content in json_contents:
        exp = json.loads(json_content)
        exps.append((exp["func_stack"], exp["config_stack"]))
    

uploader_ui = FileUpload(
    accept='.json', # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)

uploader_ui.observe(lambda _: load_exp(uploader_ui.data, exps), names='_counter')

display(uploader_ui)

In [ ]:
input_image = depth_img # rgb_img
results = []

for exp in exps:
    result = apply_functions(input_image, *exp)
    results.append(result)

In [ ]:
show_shapes(input_image, results[0], shapeIdx=0, cmap=None)

In [ ]:
show_shapes(rgb_img.copy(), results[0], shapeIdx=0, cmap=None)

In [ ]:
# Merging Canny detector result with Hough line detector result
horizontal = results[0]
vertical = convert_coord2mask(horizontal.shape, results[1][0])
merged = horizontal | vertical
show_image(merged, cmap="gray")

# Aside: Experimental

In [ ]:
tmp1 = create_template(size=20, position=1)
# ...
templates = [tmp1]

In [ ]:
# Fit a line to inlier detections
pixel_coords = []
points = []

for template in templates:
    pixels, corners, outliers = apply_template_matching(depth_arr.astype(np.float32), 
                                                        template,
                                                        y_step=25,
                                                        n_contours=200,
                                                        fit_type="line",
                                                        verbose=1)
    pixel_coords.append(pixels)
    points.append((corners, outliers))


plt.figure(figsize=(10,10))
plt.imshow(depth_img)
for corners, outliers in points:
    plt.scatter(corners[:,1], corners[:,0], color="black", marker="o")
    plt.scatter(outliers[:,1], outliers[:,0], color="purple", marker="^")
plt.show()

mask = np.zeros(depth_arr.shape, dtype=np.bool)
for p in pixel_coords:
    tmp = convert_coord2mask(depth_arr.shape, p)
    mask = mask | tmp
    
show_overlay(rgb_img, mask)
show_overlay(depth_img, mask)

In [ ]:
# Fit a curve (2nd degree polynomial) to inlier detections
pixel_coords = []
points = []

for template in templates:
    pixels, corners, outliers = apply_template_matching(depth_arr.astype(np.float32), 
                                                        template,
                                                        y_step=25,
                                                        n_contours=200,
                                                        fit_type="curve",
                                                        verbose=1)
    pixel_coords.append(pixels)
    points.append((corners, outliers))


plt.figure(figsize=(10,10))
plt.imshow(depth_img)
for corners, outliers in points:
    plt.scatter(corners[:,1], corners[:,0], color="black", marker="o")
    plt.scatter(outliers[:,1], outliers[:,0], color="purple", marker="^")
plt.show()

mask = np.zeros(depth_arr.shape, dtype=np.bool)
for p in pixel_coords:
    tmp = convert_coord2mask(depth_arr.shape, p)
    mask = mask | tmp
    
show_overlay(rgb_img, mask)
show_overlay(depth_img, mask);

In [ ]:
size = 30
mask = create_template(size)
plt.figure(figsize=(10,10))
plt.imshow(mask, cmap="gray", extent=[0, mask.shape[1], mask.shape[0], 0])
plt.xticks(np.arange(mask.shape[1]+1))
plt.yticks(np.arange(mask.shape[0]+1))
plt.show()
# np.save(f"{size}x{size}", mask)

In [ ]:
contours = find_contours(output, 0.8)
plt.figure(figsize=(10,10))
plt.imshow(output)
for contour in contours:
    plt.plot(contour[:, 1], contour[:, 0], linewidth=2)

plt.show()

convex_hull = convex_hull_image(output)
show_image(convex_hull, cmap="gray")

In [ ]:
from sklearn.linear_model import LinearRegression

choice = np.random.randint(0, 640, 5)
print("Choice:", choice)

arr = np.rot90(depth_arr, axes=(1,0))

X = np.arange(479, -1, -1)
y = np.divide(1, arr, out=np.zeros_like(arr), where=arr!=0)

# m = -0.0027798553391156096
# b = 1.3734837318132027
model = LinearRegression()
m = 0
b = 0
for i, y_vec in enumerate(y):
    model.fit(X.reshape(-1,1), y_vec.reshape(-1,1))    
    m += model.coef_.item()
    b += model.intercept_.item()

m /= i
b /= i
print("m:", m, "b:", b)
for y_vec in y[choice]:
    plt.plot(X, y_vec)

y_pred = m * X + b
plt.plot(X, y_pred, color="black", linestyle="--", linewidth=2)
plt.xlim(480, 0)
plt.show()

In [ ]:
contour_masks = []
start_depth = 0.8
step = 0.1
width = 0.02
n_contours = 10
dy = estimate_dy(start_depth, width) # dy to be kept constant
for i in range(n_contours):
    min_depth = start_depth + (step * i)
    ddepth = estimate_ddepth(min_depth, dy)
    print("ddepth:", ddepth)
    max_depth = min_depth + ddepth
    print(f"Contour-{i}: ({min_depth:.2f}-{max_depth:.2f})")
    contour_mask = (depth_arr >= min_depth) & (depth_arr <= max_depth)
    show_image(contour_mask, cmap="gray")

In [ ]:
# Generate sample data
y = np.random.randint(0, 480, (200, 1))
x1 = np.random.randint(0, 250, (10, 1))
x2 = np.random.randint(250, 450, (180, 1))
x3 = np.random.randint(450, 640, (10, 1))
x = np.vstack((x1,x2,x3))

# Cluster corners horizontally
corners = np.hstack((y,x))
inlier_corners = cluster_along_x(corners, verbose=3)

In [ ]:
p_slope = {"m": 1, "b": 160, "type": "slope"}
p_hough = {"theta": np.pi/2, "rho": 120, "type": "hough"}
y = np.random.randint(0,480,(10,1))
x = np.random.randint(0,640,(10,1))
corners = np.stack((y,x), axis=2)
cv_corners = np.stack((x,y), axis=2)
vp = cv2.fitLine(cv_corners, cv2.DIST_HUBER, 0, 0.01, 0.01).ravel()
p_sym = {"v": vp[0:2], "p": vp[2:4], "type": "symmetric"}
line_pixels1 = compute_visible_pixels(depth_arr.shape, p_slope)
line_pixels2 = compute_visible_pixels(depth_arr.shape, p_sym)
line_pixels4 = compute_visible_pixels(depth_arr.shape, p_hough)
line_mask1 = convert_coord2mask(depth_arr.shape, line_pixels1)
line_mask2 = convert_coord2mask(depth_arr.shape, line_pixels2)
line_mask4 = convert_coord2mask(depth_arr.shape, line_pixels4)
show_image(line_mask1, cmap="gray")
show_image(line_mask2, cmap="gray")
show_image(line_mask4, cmap="gray")

In [ ]:
# Interactive Hough line visualization
def draw_line(rho=240, theta=45):
    shape = (480, 640)
    top = 0
    bottom = shape[0] - 1
    left = 0
    right = shape[1] - 1
    theta = np.deg2rad(theta)
    
    # When theta is multiples of pi/2, set endpoints manually.
    if theta == np.pi/2:
        p1 = np.array([rho, left])
        p2 = np.array([rho, right])
    # Otherwise, use formula
    else:
        p1 = np.array([top, rho/np.cos(theta)])
        p2 = np.array([bottom, (rho-bottom*np.sin(theta))/np.cos(theta)])

    print("sin(theta)==0:", np.isclose(0, np.sin(theta)))
    print("cos(theta)==0:", np.isclose(0, np.cos(theta)))
    print("Start:", p1)
    print("End:", p2)

    pixel_coords = line(*np.rint(p1).astype(np.int32), *np.rint(p2).astype(np.int32))
    
    # Merge coordinates in a matrix
    pixel_coords = np.stack(pixel_coords, axis=1)
    
    # Take coordinates in the 1st quadrant
    pixel_coords = pixel_coords[np.min(pixel_coords, axis=1)>=0]
    
    # Take coordinates in the visible region
    visible = (pixel_coords[:,0] < shape[0]) & (pixel_coords[:,1] < shape[1])
    pixel_coords = pixel_coords[visible]
    
    line_image = np.zeros(shape, dtype=np.uint8)
    line_image[pixel_coords[:, 0], pixel_coords[:, 1]] = 255
    plt.figure(figsize = (10, 10))
    plt.imshow(line_image)
    
interactive(draw_line, rho=(0, 750), theta=(0, 180))

# for angle in range(0, 181, 5):
#     draw_line(rho=240, theta=angle)

In [ ]:
# Draw a convex polygon enclosing a set of points
orig = depth_img
copy1 = orig.copy()
copy2 = orig.copy()

mask1 = np.zeros(copy1.shape, dtype=np.uint8)
mask2 = np.zeros(copy2.shape, dtype=np.uint8)
roi_corners = np.array([[(0, 0), (100,100), (50,0)]], dtype=np.int32)
C = copy.shape[2]
white = (255,) * C
cv2.fillPoly(mask1, roi_corners, color=white)
cv2.fillConvexPoly(mask2, roi_corners, color=white)
# from Masterfool: use cv2.fillConvexPoly if you know it's convex
image1 = cv2.bitwise_and(copy1, mask1)
image2 = cv2.bitwise_and(copy2, mask2)

show_image(image1)
show_image(image2)

In [ ]:
# Inpainting
target = depth_img.copy()
mask = convert_grayscale(target, visualize=False)
mask = apply_threshold(mask, visualize=False, thresh=0, maxval=1, type=1)

show_image(mask, cmap="gray")

inpainted = cv2.inpaint(target, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

show_image(depth_img)
show_image(inpainted)

 # Help

## Coordinate Conventions of NumPy and OpenCV

Let us consider an arbitrary point P in the OpenCV coordinate system whose coordinates are (x, y). The same point in the NumPy image coordinate system is given by the coordinates (c, r).

Assume that we need to manipulate the image at point P and that the image is stored in the NumPy array img. OpenCV functions typically expect us to provide the coordinate information in the order **(x, y)** for its image manipulation functions. However, to access the element at point P using NumPy, we need to do **img[r, c]**.

### OpenCV Coordinates
---
![1st](figures/opencv.png "OpenCV Coordinates")

### NumPy Coordinates
---
![2nd](figures/numpy.png "NumPy Coordinates")

## cv2.COLOR_XXX

[cv2.COLOR_XXX](https://vovkos.github.io/doxyrest-showcase/opencv/sphinx_rtd_theme/enum_cv_ColorConversionCodes.html)

## cv2.THRESH_XXX

[Thresholding](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html)

[cv2.THRESH_XXX](https://vovkos.github.io/doxyrest-showcase/opencv/sphinx_rtd_theme/enum_cv_ThresholdTypes.html)

## cv2.cvtColor

In [ ]:
help(cv2.cvtColor)

## cv2.equalizeHist

In [ ]:
help(cv2.equalizeHist)

## cv2.threshold

In [ ]:
help(cv2.threshold)

## cv2.adaptiveThreshold

In [ ]:
help(cv2.adaptiveThreshold)

## cv2.filter2D

In [ ]:
help(cv2.filter2D)

## cv2.blur

In [ ]:
help(cv2.blur)

## cv2.medianBlur

In [ ]:
help(cv2.medianBlur)

## cv2.bilateralFilter

In [ ]:
help(cv2.bilateralFilter)

## cv2.GaussianBlur

In [ ]:
help(cv2.GaussianBlur)

## cv2.Sobel

In [ ]:
help(cv2.Sobel)

## cv2.Laplacian

In [ ]:
help(cv2.Laplacian)

## cv2.Canny

All pixels having intensity between 1st and 2nd thresholds are flagged as weak and the Hysteresis mechanism will identify the ones that could be considered as strong and the ones that are considered as non-relevant.

In [ ]:
help(cv2.Canny)

## cv2.findContours

[Convex Hull](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html#convex-hull)

[Fitting a Line](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html#fitting-a-line)

[Mask and Pixel Points](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_properties/py_contour_properties.html#mask-and-pixel-points)

[Match Shapes](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contours_more_functions/py_contours_more_functions.html#match-shapes)

In [ ]:
help(cv2.findContours)

## cv2.HoughLines

* If the line is passing below the origin, it will have a positive rho and an angle less than 180. If it is going above the origin, instead of taking an angle greater than 180, the angle is taken less than 180, and rho is taken negative. 
* Any vertical line will have 0 degree and horizontal lines will have 90 degree.

[Hough Transform Theory](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html#theory)

In [ ]:
help(cv2.HoughLines)

## cv2.inpaint

In [ ]:
help(cv2.inpaint)

## skimage.filters.farid

In [ ]:
help(filters.farid)
help(filters.farid_v)
help(filters.farid_h)

## skimage.filters.roberts

In [ ]:
help(filters.roberts)

## cv2.goodFeaturesToTrack

In [ ]:
help(cv2.goodFeaturesToTrack)

## cyvlfeat.sift.sift

In [ ]:
help(cyvlfeat.sift.sift)

## cyvlfeat.sift.dsift

In [ ]:
help(cyvlfeat.sift.dsift)

## Torchvision Models

[Available Model List](https://pytorch.org/docs/stable/torchvision/models.html)